In [10]:
import pandas as pd

df = pd.read_sas("P_HEQ.XPT")

df

,SEQN,HEQ010,HEQ020,HEQ030,HEQ040
0,109264.0,2.0,NaN,2.0,NaN
1,109266.0,2.0,NaN,2.0,NaN
2,109267.0,2.0,NaN,2.0,NaN
3,109268.0,2.0,NaN,2.0,NaN
4,109270.0,2.0,NaN,2.0,NaN
...,...,...,...,...,...
13213,124817.0,2.0,NaN,2.0,NaN
13214,124818.0,2.0,NaN,2.0,NaN
13215,124820.0,2.0,NaN,2.0,NaN
13216,124821.0,2.0,NaN,2.0,NaN
